<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/JFriedel_IOT_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Joseph Friedel                          AAI 530                     Assignment 4                             3 February 2025

Long Short Term Memory Networks for IoT Prediction

RNNs and LSTM models are very popular neural network architectures when working with sequential data, since they both carry some "memory" of previous
inputs when predicting the next output. In this assignment we will continue to work with the Household Electricity Consumption dataset and use an LSTM
model to predict the Global Active Power (GAP) from a sequence of previous GAP readings. You will build one model following the directions in this
notebook closely, then you will be asked to make changes to that original model and analyze the effects that they had on the model performance. You will
also be asked to compare the performance of your LSTM model to the linear regression predictor that you built in last week's assignment.

General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading
it.

One sign of mature code is conforming to a style guide. We recommend the Google Python Style Guide. If you use a different style guide, please include a
cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final
submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential import statements and make sure
that all such statements are moved into the designated cell.

When you save your notebook as a pdf, make sure that all cell output is visible (even error messages) as this will aid your instructor in grading your
work.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions
to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. Make sure to answer every question marked with a
Q: for full credit.

In [ ]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Setting seed for reproducibility
np.random.seed(1234)
PYTHONHASHSEED = 0

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import pad_sequences

In [ ]:
#use this cell to import additional libraries or define helper functions
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error as mse
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences

Load and prepare your data

We'll once again be using the cleaned household electricity consumption data from the previous two assignments. I recommend saving your dataset by
running df.to_csv("filename") at the end of assignment 2 so that you don't have to re-do your cleaning steps. If you are not confident in your own
cleaning steps, you may ask your instructor for a cleaned version of the data. You will not be graded on the cleaning steps in this assignment, but
some functions may not work if you use the raw data.

Unlike when using Linear Regression to make our predictions for Global Active Power (GAP), LSTM requires that we have a pre-trained model when our
predictive software is shipped (the ability to iterate on the model after it's put into production is another question for another day). Thus, we will
train the model on a segment of our data and then measure its performance on simulated streaming data another segment of the data. Our dataset is very
large, so for speed's sake, we will limit ourselves to 1% of the entire dataset.

TODO: Import your data, select the a random 1% of the dataset, and then split it 80/20 into training and validation sets (the test split will come from
the training data as part of the tensorflow LSTM model call). HINT: Think carefully about how you do your train/validation split--does it make sense to
randomize the data?

In [ ]:
# Display the last few rows of the DataFrame to confirm it loaded correctly
print(df.tail())

In [ ]:
df.shape

In [ ]:
# Save a copy of df before reducing it
df_clean = df.copy()

In [ ]:
#create your training and validation sets here
#assign size for data subset
num_rows = int(len(df) * 0.01)

#take random data subset
df = df.sample(n=num_rows, random_state=42).sort_index().copy()

In [ ]:
df.shape

In [ ]:
#split data subset 80/20 for train/validation
# Determine the number of rows for training and validation
num_rows = len(df)
num_val = int(num_rows * 0.2)  # 20% for validation
num_train = num_rows - num_val  # Remaining 80% for training

# Randomly sample validation indices, then sort them sequentially
val_indices = df.sample(n=num_val, random_state=42).index
val_indices = sorted(val_indices)  # Ensure sequential order

# Create validation and training DataFrames
val_df = df.loc[val_indices].copy()
train_df = df.drop(val_indices).copy()

In [ ]:
#reset the indices for cleanliness
train_df = train_df.reset_index()
val_df = val_df.reset_index()

In [ ]:
print(train_df.head())

Next we need to create our input and output sequences. In the lab session this week, we used an LSTM model to make a binary prediction, but LSTM models
are very flexible in what they can output: we can also use them to predict a single real-numbered output (we can even use them to predict a sequence of
outputs). Here we will train a model to predict a single real-numbered output such that we can compare our model directly to the linear regression model
from last week.

TODO: Create a nested list structure for the training data, with a sequence of GAP measurements as the input and the GAP measurement at your predictive
horizon as your expected output

In [ ]:
seq_arrays = []
seq_labs = []

In [ ]:
# we'll start out with a 30 minute input sequence and a 5 minute predictive horizon
# we don't need to work in seconds this time, since we'll just use the indices instead of a unix timestamp
seq_length = 30
ph = 5

feat_cols = ['Global_active_power']

In [ ]:
#create list of sequence length GAP readings
for i in range(len(train_df) - seq_length - ph):
    # Extract input sequence (last `seq_length` readings)
    seq_arrays.append(train_df[feat_cols].iloc[i : i + seq_length].values)

    # Extract expected output (GAP measurement at `ph` steps ahead)
    seq_labs.append(train_df['Global_active_power'].iloc[i + seq_length + ph])

In [ ]:
#convert to numpy arrays and floats to appease keras/tensorflow
seq_arrays = np.array(seq_arrays, dtype = object).astype(np.float32)
seq_labs = np.array(seq_labs, dtype = object).astype(np.float32)

In [ ]:
# Assertions to ensure correct shape
assert seq_arrays.shape == (len(train_df) - seq_length - ph, seq_length, len(feat_cols))
assert seq_labs.shape == (len(train_df) - seq_length - ph,)

In [ ]:
# Print shape of input sequences
seq_arrays.shape

Q: What is the function of the assert statements in the above cell? Why do we use assertions in our code?

A: The assert statements are used for debugging and validation. They make sure that the seq_arrays and seq_labs variables have the needed shape before
being used by the LSTM model.  It is used to find errors early in the program.  If seq_arrays and seq_labs have the wrong shapes, the assertions will
raise an error early on.  It ensures the correct data structures for LSTM:  seq_arrays is expected to be a 3D NumPy array and seq_labs is expected
to be a 1D NumPy array.  If the shapes do not match the expected format an Assertion Error is raised.

Model Training

We will begin with a model architecture very similar to the model we built in the lab session. We will have two LSTM layers, with 5 and 3 hidden units
respectively, and we will apply dropout after each LSTM layer. However, we will use a LINEAR final layer and MSE for our loss function, since our output
is continuous instead of binary.

TODO: Fill in all values marked with a ?? in the cell below

In [ ]:
# Define path to save model
model_path = 'LSTM_model1.keras'

# Build the network
# Define the number of input features and output dimensions
nb_features = len(feat_cols)  # Number of features (should be 1 for GAP readings)
nb_out = 1  # Single real-valued output (next GAP reading at predictive horizon)

# Build the LSTM network
model = Sequential()

# Add first LSTM layer
model.add(LSTM(
    input_shape=(seq_length, nb_features),  # (time steps, features)
    units=50,  # Number of LSTM neurons
    return_sequences=True))  # Return sequences for stacking layers
model.add(Dropout(0.2))  # Prevent overfitting

# Add second LSTM layer
model.add(LSTM(
    units=50,  # Number of neurons in second layer
    return_sequences=False))  # Final LSTM layer does not return sequences
model.add(Dropout(0.2))

# Add fully connected layer for output
model.add(Dense(units=nb_out))  # Predict a single value
model.add(Activation('linear'))  # Use a linear activation function for regression

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mse'])

# Print model summary
print(model.summary())

# Fit the model
history = model.fit(seq_arrays, seq_labs,  # Training data (X, y)
                    epochs=100,
                    batch_size=500,
                    validation_split=0.05,  # Use 5% of training data for validation
                    verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )

# List all data in history
print(history.history.keys())

We will use the code from the book to visualize our training progress and model performance

In [ ]:
# summarize history for Loss/MSE
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss/MSE')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig_acc.savefig("LSTM_loss1.png")

Validating our model

Now we need to create our simulated streaming validation set to test our model "in production". With our linear regression models, we were able to
begin making predictions with only two datapoints, but the LSTM model requires an input sequence of seq_length to make a prediction. We can get around
this limitation by "padding" our inputs when they are too short.

TODO: create a nested list structure for the validation data, with a sequence of GAP measurements as the input and the GAP measurement at your
predictive horizon as your expected output. Begin your predictions after only two GAP measurements are available, and check out this keras function to
automatically pad sequences that are too short.

Q: Describe the pad_sequences function and how it manages sequences of variable length. What does the "padding" argument determine, and which setting
makes the most sense for our use case here?

A: Format:

from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequences = pad_sequences(sequences,
                                 maxlen=None,
                                 dtype='int32',
                                 padding='pre',
                                 truncating='pre',
                                 value=0.0)

Argument "sequences" is a List of lists, with each sublist being a sequence of vary length.
Argument "maxlen" is the maximum sequence length, which results in longer sequences being truncated, while shorter sequences become padded.
The "dtype" is the Data type, like float32.
Variable "padding" tells where to add padding: 'pre' is before and 'post' is after.
Variable "truncating" tells	where to truncate longer sequences: 'pre' is at the beginning and 'post' is at the end.
The variable "value" is the value used for padding, and defaults to 0.0.

When using "pad_sequences" short sequences are padded to reach maxlen.  Long sequences are shortened if they exceed maxlen.  This ensures all
    sequeces have the same length for deep learning model batch processing.

I'll use padding = 'pre' because I'm working with time-series forcasting, using LSTM.   LSTM uses "past to present" sequence processing, so using
    padding in the beqinning makes sure that most recent data is at the end of the sequence.  The most recent data is the most useful for prediction.

In [ ]:
val_arrays = []
val_labs = []

# Create list of GAP readings starting with a minimum of two readings
for i in range(2, len(val_df) - ph):  # Start from index 2 to ensure at least two readings
    # Extract input sequence (from the start to current index)
    val_arrays.append(val_df[feat_cols].iloc[:i].values)

    # Extract expected output (GAP measurement at predictive horizon)
    val_labs.append(val_df['Global_active_power'].iloc[i + ph])

# Use the pad_sequences function on your input sequences
# remember that we will later want our datatype to be np.float32
val_arrays = pad_sequences(val_arrays, maxlen=seq_length, padding='pre', dtype='float32')

# Convert labels to numpy arrays and floats to appease keras/tensorflow
val_labs = np.array(val_labs, dtype = object).astype(np.float32)

We will now run this validation data through our LSTM model and visualize its performance like we did on the linear regression data.

In [ ]:
# Generate predictions on validation data
predictions = model.predict(val_arrays)

# Create a time axis for plotting
time_axis = range(len(val_labs))  # X-axis representing time

# Plot actual vs. predicted values
plt.figure(figsize=(12, 6))
plt.plot(time_axis, val_labs, label="Actual Values", color='blue', linestyle='solid')
plt.plot(time_axis, predictions, label="Predicted Values", color='red', linestyle='dashed')

# Labels and title
plt.xlabel("Time Steps")
plt.ylabel("Global Active Power (GAP)")
plt.title("LSTM Predictions vs. Actual GAP Values")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [ ]:
scores_test = model.evaluate(val_arrays, val_labs, verbose=2)
print('\nMSE: {}'.format(scores_test[1]))

y_pred_test = model.predict(val_arrays)
y_true_test = val_labs

test_set = pd.DataFrame(y_pred_test)
test_set.to_csv('submit_test.csv', index = None)

# Plot the predicted data vs. the actual data
# We will limit our plot to the first 500 predictions for better visualization
fig_verify = plt.figure(figsize=(10, 5))
plt.plot(y_pred_test[-500:], label = 'Predicted Value')
plt.plot(y_true_test[-500:], label = 'Actual Value')
plt.title('Global Active Power Prediction - Last 500 Points', fontsize=22, fontweight='bold')
plt.ylabel('value')
plt.xlabel('row')
plt.legend()
plt.show()
fig_verify.savefig("model_regression_verify.png")

Q: How did your model perform? What can you tell about the model from the loss curves? What could we do to try to improve the model?

A: The model did fair:  it landed within the data range, and matched the real data in upward and downward slopes, but the model
did not match the range of the actual data.  The model ranged from about o.5 to about 1.5 while the actual data ranged from about 0.
to 6.   From the loss curves, I could be underfitting, because the model is too simple or I could have a too high learning rate, causing the
optimizer to miss important patterns or I could have too little training, so the model hasn't learned trends well enough.  So I could increase
the units in the LSTM layers or add more LSTM layers, or reduce the dropout, if underfitting is the problem.  Or I could reduce the learnin rate, or
change the optimizer if the learning rate is the problem.  If training is the issue, I can increase epochs.

Model Optimization

Now it's your turn to build an LSTM-based model in hopes of improving performance on this training set. Changes that you might consider include:

Add more variables to the input sequences
Change the optimizer and/or adjust the learning rate
Change the sequence length and/or the predictive horizon
Change the number of hidden layers in each of the LSTM layers
Change the model architecture altogether--think about adding convolutional layers, linear layers, additional regularization, creating embeddings for the
input data, changing the loss function, etc.

There isn't any minimum performance increase or number of changes that need to be made, but I want to see that you have tried some different things.
Remember that building and optimizing deep learning networks is an art and can be very difficult, so don't make yourself crazy trying to optimize for
this assignment.

Q: What changes are you going to try with your model? Why do you think these changes could improve model performance?

Well I'll try lowering the Learning Rate to try to catch finer patterns. Then I'll try increasing the LSTM complexity,
like adding layers, in case my model is too simple.  Then I'll ensure the data is shaped correctly, In case I'm inputting
the data incorrectly, and that's causing the poor results.

# play with your ideas for optimization here

# show me how one or two of your different models perform
# using the code from the "Validating our model" section above

# 1) Lowering the learning rate

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

In [ ]:
# Fit the model
history = model.fit(seq_arrays, seq_labs,  # Training data (X, y)
                    epochs=100,
                    batch_size=500,
                    validation_split=0.05,  # Use 5% of training data for validation
                    verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )

In [ ]:
val_arrays = []
val_labs = []

# Create list of GAP readings starting with a minimum of two readings
for i in range(2, len(val_df) - ph):  # Start from index 2 to ensure at least two readings
    # Extract input sequence (from the start to current index)
    val_arrays.append(val_df[feat_cols].iloc[:i].values)

    # Extract expected output (GAP measurement at predictive horizon)
    val_labs.append(val_df['Global_active_power'].iloc[i + ph])

# Use the pad_sequences function on your input sequences
# remember that we will later want our datatype to be np.float32
val_arrays = pad_sequences(val_arrays, maxlen=seq_length, padding='pre', dtype='float32')

# Convert labels to numpy arrays and floats to appease keras/tensorflow
val_labs = np.array(val_labs, dtype = object).astype(np.float32)

In [ ]:
scores_test = model.evaluate(val_arrays, val_labs, verbose=2)
print('\nMSE: {}'.format(scores_test[1]))

y_pred_test = model.predict(val_arrays)
y_true_test = val_labs

test_set = pd.DataFrame(y_pred_test)
test_set.to_csv('submit_test.csv', index = None)

# Plot the predicted data vs. the actual data
# We will limit our plot to the first 500 predictions for better visualization
fig_verify = plt.figure(figsize=(10, 5))
plt.plot(y_pred_test[-500:], label = 'Predicted Value')
plt.plot(y_true_test[-500:], label = 'Actual Value')
plt.title('Global Active Power Prediction - Last 500 Points', fontsize=22, fontweight='bold')
plt.ylabel('value')
plt.xlabel('row')
plt.legend()
plt.show()
fig_verify.savefig("model_regression_verify.png")

No change :(

# 2) Increasing LSTM complexity:

In [ ]:
# Ensure the input shape of your data is (samples, timesteps, features)
# Make sure seq_arrays is shaped (samples, seq_length, nb_features)
seq_arrays = np.reshape(seq_arrays, (seq_arrays.shape[0], seq_arrays.shape[1], nb_features))

# Build the model
model = Sequential()

# Add the first LSTM layer
model.add(LSTM(
    units=100,              # Increase neurons to capture more complexity
    return_sequences=True,  # Keep sequences for the next LSTM layer
    input_shape=(seq_length, nb_features)))  # Correct input shape (timesteps, features)
model.add(Dropout(0.1))  # Reduce dropout slightly

# Add the second LSTM layer
model.add(LSTM(
    units=100,  # Increase neurons in the second LSTM layer
    return_sequences=False))  # Don't return sequences from the last LSTM
model.add(Dropout(0.1))  # Reduce dropout

# Add Dense layer for output
model.add(Dense(units=nb_out))  # Output a single prediction (next GAP value)
model.add(Activation('linear'))  # Linear activation for regression output

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Lower learning rate
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

# Print model summary
print(model.summary())

# Fit the model with training data
history = model.fit(seq_arrays, seq_labs,  # Training data
                    epochs=200,  # Increase the number of epochs
                    batch_size=500,
                    validation_split=0.05,  # 5% of training data used for validation
                    verbose=2,
                    callbacks=[
                        keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='min'),
                        keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
                    ])

In [ ]:
val_arrays = []
val_labs = []

# Create list of GAP readings starting with a minimum of two readings
for i in range(2, len(val_df) - ph):  # Start from index 2 to ensure at least two readings
    # Extract input sequence (from the start to current index)
    val_arrays.append(val_df[feat_cols].iloc[:i].values)

    # Extract expected output (GAP measurement at predictive horizon)
    val_labs.append(val_df['Global_active_power'].iloc[i + ph])

# Use the pad_sequences function on your input sequences
# remember that we will later want our datatype to be np.float32
val_arrays = pad_sequences(val_arrays, maxlen=seq_length, padding='pre', dtype='float32')

# Convert labels to numpy arrays and floats to appease keras/tensorflow
val_labs = np.array(val_labs, dtype = object).astype(np.float32)

In [ ]:
scores_test = model.evaluate(val_arrays, val_labs, verbose=2)
print('\nMSE: {}'.format(scores_test[1]))

y_pred_test = model.predict(val_arrays)
y_true_test = val_labs

test_set = pd.DataFrame(y_pred_test)
test_set.to_csv('submit_test.csv', index = None)

# Plot the predicted data vs. the actual data
# We will limit our plot to the first 500 predictions for better visualization
fig_verify = plt.figure(figsize=(10, 5))
plt.plot(y_pred_test[-500:], label = 'Predicted Value')
plt.plot(y_true_test[-500:], label = 'Actual Value')
plt.title('Global Active Power Prediction - Last 500 Points', fontsize=22, fontweight='bold')
plt.ylabel('value')
plt.xlabel('row')
plt.legend()
plt.show()
fig_verify.savefig("model_regression_verify.png")

No change :(

3) Reshaping Data

In [ ]:
# Ensure the input shape of data is (samples, timesteps, features)
# Make sure seq_arrays is shaped (samples, seq_length, nb_features)
seq_arrays = np.reshape(seq_arrays, (seq_arrays.shape[0], seq_arrays.shape[1], nb_features))

# Build the model
model = Sequential()

# Add the first LSTM layer
model.add(LSTM(
    units=100,              # Increase neurons to capture more complexity
    return_sequences=True,  # Keep sequences for the next LSTM layer
    input_shape=(seq_length, nb_features)))  # Correct input shape (timesteps, features)
model.add(Dropout(0.1))  # Reduce dropout slightly

# Add the second LSTM layer
model.add(LSTM(
    units=100,  # Increase neurons in the second LSTM layer
    return_sequences=False))  # Don't return sequences from the last LSTM
model.add(Dropout(0.1))  # Reduce dropout

# Add Dense layer for output
model.add(Dense(units=nb_out))  # Output a single prediction (next GAP value)
model.add(Activation('linear'))  # Linear activation for regression output

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Lower learning rate
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

# Print model summary
print(model.summary())

# Fit the model with training data
history = model.fit(seq_arrays, seq_labs,  # Training data
                    epochs=200,  # Increase the number of epochs
                    batch_size=500,
                    validation_split=0.05,  # 5% of training data used for validation
                    verbose=2,
                    callbacks=[
                        keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='min'),
                        keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
                    ])

In [ ]:
val_arrays = []
val_labs = []

# Create list of GAP readings starting with a minimum of two readings
for i in range(2, len(val_df) - ph):  # Start from index 2 to ensure at least two readings
    # Extract input sequence (from the start to current index)
    val_arrays.append(val_df[feat_cols].iloc[:i].values)

    # Extract expected output (GAP measurement at predictive horizon)
    val_labs.append(val_df['Global_active_power'].iloc[i + ph])

# Use the pad_sequences function on your input sequences
# remember that we will later want our datatype to be np.float32
val_arrays = pad_sequences(val_arrays, maxlen=seq_length, padding='pre', dtype='float32')

# Convert labels to numpy arrays and floats to appease keras/tensorflow
val_labs = np.array(val_labs, dtype = object).astype(np.float32)

In [ ]:
scores_test = model.evaluate(val_arrays, val_labs, verbose=2)
print('\nMSE: {}'.format(scores_test[1]))

y_pred_test = model.predict(val_arrays)
y_true_test = val_labs

test_set = pd.DataFrame(y_pred_test)
test_set.to_csv('submit_test.csv', index = None)

# Plot the predicted data vs. the actual data
# We will limit our plot to the first 500 predictions for better visualization
fig_verify = plt.figure(figsize=(10, 5))
plt.plot(y_pred_test[-500:], label = 'Predicted Value')
plt.plot(y_true_test[-500:], label = 'Actual Value')
plt.title('Global Active Power Prediction - Last 500 Points', fontsize=22, fontweight='bold')
plt.ylabel('value')
plt.xlabel('row')
plt.legend()
plt.show()
fig_verify.savefig("model_regression_verify.png")

No change :(

Q: How did your model changes affect performance on the validation data? Why do you think they were/were not effective? If you were trying to optimize
    for production, what would you try next?

A: No change!  That makes me suspect there is not a problem with the model.  So that makes me suspect the data.  The next change I would do is take a
larger sampling of data: instead of 1%, maybe 5%.

Q: How did the models that you built in this assignment compare to the linear regression model from last week? Think about model performance and other
IoT device considerations; Which model would you choose to use in an IoT system that predicts GAP for a single household with a 5-minute predictive
horizon, and why?

A: I thought the Linear Regression prediction followed the data beter and the MSE was lower.  For a 5-minute horizon, I would use
the LSTM model.  I think LSTM is more capable to showing the highly varying, complex  patterns of GAP than LR.  LR would be better for modeling
data with less variance over time.